# Practical 1: Position and Orientation

If you are running the notebook **locally**, please set the variable *running_locally* to true. If you are running the notebook **on Colab**, please set *running_locally* to false. 
You may also find these commands useful when using the notebook: 
*   "Ctrl" + "/" to comment/uncomment
*   "Shift" + "Enter" to run the block
*   "Ctrl" + "M" + "." to restart the kernel/runtime

In this code block, we install the necessary library and append the path for supporting scripts.


In [ ]:
running_locally = False

In [ ]:
import sys
import os
if running_locally:
  !pip install plotly
  sys.path.append('Practical1_Support')
else:
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  sys.path.append('/content/gdrive/MyDrive/ECE4078_Practical/Week01/Practical1_Support')
  !ls '/content/gdrive/MyDrive/ECE4078_Practical/Week01/Practical1_Support'

Then, we import the required dependencies. This step is required every time you close the notebook for both running the notebook **locally and on colab)**.

In [ ]:
# this line tells Jupyter that we are rendering plots in "inline" style
%matplotlib inline

# import libraries
import ipywidgets as widgets 
from ipywidgets import interact, interactive, interact_manual, interactive_output
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import Math, display, clear_output
import pandas as pd
from matplotlib.patches import FancyArrowPatch
import matplotlib.transforms as transforms

# import supporting scripts
from NotebookCheck import *
from RotationSupport2D import *
from RotationSupport3D import *

# Testing the Notebook First

To make sure that your notebook instance has been created correctly, please execute the code below. 

You should see the plot change as you move the slider. 

**FLUX Question:** What is the phrase shown when Omega ($\omega$) =10 at the end?

In [ ]:
notebook_plot = NotebookChecker()
display(notebook_plot)

## 1. Rotations in 2D

We define a 2D coordinate frame to represent our robot $(\boldsymbol{x}_1, \boldsymbol{y}_1)$ with respect to the world frame $(\boldsymbol{x}_0, \boldsymbol{y}_0)$. 

Given an angle $\theta$, we describe the relation between the world and robot's frames using the following rotation matrix $R_{01} = \begin{vmatrix}cos\theta & -sin\theta \\sin\theta & cos\theta \end{vmatrix}$.

Below we show how the robot's frame (green and red arrows) changes relative to the world frame (black) as the angle $\theta$ increases or decreases.

**Interaction**:
- Move the slider to change the rotation angle around the origin

In [ ]:
# here we define the interactive components
import matplotlib.pyplot as plt

theta_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1,
                                   description=r'Theta (rad)',continuous_update=False)

**Note**

You may need to wait for the graph to refresh.

In [ ]:
def rotate(change):
    r_theta = change*np.pi
    cos_theta = np.cos(r_theta)
    sin_theta = np.sin(r_theta)
    
    rot_mat = np.array([[cos_theta, -sin_theta],
                       [sin_theta, cos_theta]])
    
    print('R = \n',np.round(rot_mat,2))

    return rot_mat
    
def main(change):
    frame2D = Rotation2D()
    rot_mat = rotate(change)
    frame2D.update_frame({"rotation": rot_mat}, frame2D.fig, frame2D.robot_ax, frame2D.ax_trans)
    plt.show()

interactive_plot = interactive(main, change=theta_slider)
display(interactive_plot)

**FLUX Question:** What is the value of the rotation matrix when $\theta=\frac{\pi}{2}$ (0.5 in the slider)?

## 2. Rotations in 3D

Let us now extend the definition of rotations to a 3-dimensional world.

Recall that $R_{x}(\theta) = \begin{vmatrix}1 & 0 & 0 \\ 0 & cos\theta & -sin\theta \\ 0 & sin\theta & cos\theta \end{vmatrix}$, $R_{y}(\theta) = \begin{vmatrix} cos\theta & 0 & sin\theta \\ 0 & 1 & 0 \\ -sin\theta & 0 & cos\theta \end{vmatrix}$, and $R_{z}(\theta) = \begin{vmatrix}cos\theta & -sin\theta & 0 \\ sin\theta & cos\theta & 0 \\ 0 & 0 & 1 \end{vmatrix}$

**Interaction**:
- Move the sliders to change the rotation angle around each axis
- Click on a button to change the composition order of the rotation matrices

**TO DO**:
- Complete the definition of $R_y(\theta)$
- Implement the $x$-$z$-$y$ and $z$-$x$-$y$ rotation orders

In [ ]:
# we again setup the interaction component; this time we use a different graphics library
import plotly.graph_objects as plt 

mode_btn = widgets.ToggleButtons(
  options=['x-z-y', 'z-x-y', 'y-z-x'],
  description='Transform:',
)


In [ ]:
def rotate3D(mode, theta_x, theta_y, theta_z):  
    
    x_theta = theta_x*np.pi
    y_theta = theta_y*np.pi
    z_theta = theta_z*np.pi
        
    rot_x = np.eye(3)
    rot_y = np.eye(3)
    rot_z = np.eye(3)
    
    rot_x[1:, 1:] = [[np.cos(x_theta), -np.sin(x_theta)],
                     [np.sin(x_theta), np.cos(x_theta)]]        
    rot_z[0:2, 0:2] = [[np.cos(z_theta), -np.sin(z_theta)],
                      [np.sin(z_theta), np.cos(z_theta)]]
    
    #-------------TO DO 1: Update rot_y ---------------
    rot_y[0, :] = [0, 0, 0]
    rot_y[2, :] = [0, 0, 0]
    
    if mode == 'x-z-y':
    # ------------TO DO 2: Complete rotation order x-z-y----------------
        rot_mat = np.eye(3)
    
    elif mode == 'z-x-y':
    # ------------TO DO 3: Complete rotation order z-x-y----------------
        rot_mat = np.eye(3)
    
    else:
        rot_mat = rot_x @ rot_z @ rot_y


    print('R ', np.round(rot_mat,decimals=2))
    xdata, ydata, zdata = transform3Daxes(rot_mat,trans_enabled=False)

    return xdata, ydata, zdata

disc = 0.01 # change this value to modify the slider step size
display(mode_btn)


@widgets.interact(theta_x=(-2, 2, disc), theta_y=(-2, 2, disc), theta_z=(-2, 2, disc))
def plot(theta_x, theta_y, theta_z, grid=True):
  mode = mode_btn.value
  xdata, ydata, zdata =rotate3D(mode, theta_x, theta_y, theta_z)  
  fig = plt.Figure()
  fig = plot3DRot(fig, xdata, ydata, zdata, 'Figure 2')
  fig.show()


**FLUX Question**: Are the rotations expressed in fixed or successive frame? 

# 3. Homogeneous Transformations in 3D

Let us now combine rotations and translations in a 3-dimensional world.

Recall that $T_{01} = \begin{vmatrix} R_{01} & \boldsymbol{d}^0_1\\ 0 & 1\end{vmatrix}$, where $R_{01}$ and $\boldsymbol{d}^0_1$ correspond to the rotation and displacement of the robot frame, i.e, *frame 1*, relative to the world frame, i.e., *frame 0*.

**Interaction**:
- Use the button (i.e. x, y, z, transform) and the sliders to visualize the rotation and displacement of the robot frame relative to the corresponding axis in the world frame. Note: if x transform is selected, slider values not associated with x will reset.

**TO DO**:
- Complete the definition of $T_{01}$ along the $z$-axis

In [ ]:
mode_btn = widgets.ToggleButtons(
  options=['x transform', 'y transform', 'z transform'],
  description='Transform:',
)

plt_btn = widgets.Button(
  description='Plot',
)

# X axis widgets
x_theta_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'theta_x (rad)',
                                     continuous_update=False)
x_disp_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'd_x',
                                     continuous_update=False)
# Y axis widgets
y_theta_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'theta_y (rad)',
                                     continuous_update=False)
y_disp_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'd_y',
                                     continuous_update=False)
# Z axis widgets
z_theta_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'theta_z (rad)',
                                     continuous_update=False)
z_disp_slider = widgets.FloatSlider(value=0, min=-2, max=2, step=.1, description=r'd_z',
                                     continuous_update=False)

def reset_sliders(x=False, y=False, z=False):
  if x:
    x_theta_slider.value = 0
    x_disp_slider.value = 0
    
  if y:
    y_theta_slider.value = 0
    y_disp_slider.value = 0
    
  if z:
    z_theta_slider.value = 0
    z_disp_slider.value = 0

def transform_frame(rotation, translation):
  new_transform = np.matmul(rotation, translation)
  return new_transform

def plot_data(fig, xdata, ydata, zdata):
    fig.data = []
    fig = plot3DRot(fig, xdata, ydata, zdata)
    fig.show()

def x_transform(theta, x):
    x_theta = theta*np.pi
    trans = np.eye(4)
    rot = np.eye(4)
    
    # Assign displacement in x direction
    trans[0,-1] = x
    # Assign rotation around x_axis
    rot[1:3, 1:3] = [[np.cos(x_theta), -np.sin(x_theta)],
                      [np.sin(x_theta), np.cos(x_theta)]]
    tf = transform_frame(rot, trans)
    print('T \n', np.round(tf,2))
    xdata, ydata, zdata = transform3Daxes(tf,trans_enabled=True)
    reset_sliders(False, True, True)
    return xdata, ydata, zdata
    

def y_transform(theta, y):
    y_theta = theta*np.pi
    trans = np.eye(4)
    rot = np.eye(4)

    # Assign displacement in y direction
    trans[1,-1] = y
    # Assign rotation around y_axis
    rot[0, 0:3] = [np.cos(y_theta), 0, np.sin(y_theta)]
    rot[2, 0:3] = [-np.sin(y_theta), 0, np.cos(y_theta)]
    tf = transform_frame(rot, trans)
    print('T \n', np.round(tf,2))
    xdata, ydata, zdata = transform3Daxes(tf,trans_enabled=True)
    reset_sliders(True, False, True)
    return xdata, ydata, zdata

def z_transform(theta, z):
    
    z_theta = theta*np.pi
    trans = np.eye(4)
    rot = np.eye(4)
    
    # Assign displacement in z direction
    trans[2,-1] = z
    
    # Assign rotation around z_axis
    #------------------ TODO -------------------------
    #  Complete definition z-axis homogeneous transformations
    #-------------------------------------------------
    
    tf = transform_frame(rot, trans)
    print('T \n', np.round(tf,2))
    xdata, ydata, zdata = transform3Daxes(tf,trans_enabled=True)
    reset_sliders(True, True, False)
    return xdata, ydata, zdata


display(mode_btn, x_theta_slider, y_theta_slider, z_theta_slider, 
        x_disp_slider, y_disp_slider, z_disp_slider, plt_btn)
@plt_btn.on_click
def plot(grid=True):
    if mode_btn.value == 'x transform':
        xdata, ydata, zdata = x_transform(x_theta_slider.value, x_disp_slider.value)
    elif mode_btn.value == 'y transform':
        xdata, ydata, zdata = y_transform(y_theta_slider.value, y_disp_slider.value)
    else:
        xdata, ydata, zdata = z_transform(z_theta_slider.value, z_disp_slider.value)
    
    clear_output(wait=True)
    display(mode_btn, x_theta_slider, y_theta_slider, z_theta_slider, 
        x_disp_slider, y_disp_slider, z_disp_slider, plt_btn)
    fig = plt.Figure()
    fig = plot3DRot(fig, xdata, ydata, zdata, 'Figure 3')
    fig.show()